In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.signal import argrelextrema
from numba import jit
from collections import Counter
import optuna
import joblib
import json
import gc
import traceback
import os
from datetime import datetime
import tensorflow as tf

# --------------------------------------------------------------------------------
# 1. 유틸리티 함수 (수정 불필요)
# --------------------------------------------------------------------------------
def add_indicator_to_df(df_ta, indicator):
    if indicator is None: return
    if isinstance(indicator, pd.DataFrame):
        for col in indicator.columns: df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series):
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except:
        return False

# --------------------------------------------------------------------------------
# 2. 외부 데이터/파생변수 추가 함수 
# --------------------------------------------------------------------------------
import pandas as pd
import pandas_ta as ta
import numpy as np

# --------------------------------------------------------------------------------
# 1. 외부 데이터/파생변수 추가 함수
# --------------------------------------------------------------------------------
def add_kimchi_premium_proxy(df):
    """업비트/바이낸스 가격 괴리율(김프) 및 Z-score 계산"""
    df_kp = df.copy()
    target_coins = ['ETH', 'BTC', 'XRP', 'SOL', 'ADA', 'DOGE', 'AVAX', 'DOT']
    
    for coin in target_coins:
        krw_col = f'{coin}_Close'
        bin_col = f'{coin}_Bin_Close'
        
        # 두 컬럼이 모두 존재할 때만 계산
        if krw_col in df_kp.columns and bin_col in df_kp.columns:
            ratio_col = f'{coin}_KP_Ratio'
            
            # 김프 비율 계산 (업비트 / 바이낸스)
            df_kp[ratio_col] = df_kp[krw_col] / (df_kp[bin_col] + 1e-9)
            
            # Z-score (180개 봉 = 약 30일 기준)
            ma = df_kp[ratio_col].rolling(180).mean()
            std = df_kp[ratio_col].rolling(180).std()
            df_kp[f'{coin}_KP_Zscore'] = (df_kp[ratio_col] - ma) / (std + 1e-9)
            
            # 변화율
            df_kp[f'{coin}_KP_Change'] = df_kp[ratio_col].pct_change()
    
    # 불필요한 바이낸스 원본 컬럼 삭제 (선택사항)
    bin_cols = [c for c in df.columns if "_Bin_" in c]
    df_kp = df_kp.drop(columns=bin_cols, errors='ignore')

    return df_kp

def add_funding_and_onchain_features(df):
    """펀딩비 추세 및 변화율 계산"""
    df_on = df.copy()
    
    if 'fundingRate' in df_on.columns:
        df_on['FR_Abs_Signal'] = df_on['fundingRate'].abs()
        # 6개 봉(24시간) 평균 추세
        df_on['FR_Trend'] = df_on['fundingRate'].rolling(6).mean()
        df_on['FR_Change'] = df_on['fundingRate'].diff()

    return df_on

# --------------------------------------------------------------------------------
# 2. 핵심 기술적 지표 계산 함수 (4시간봉 최적화)
# --------------------------------------------------------------------------------
def calculate_technical_indicators_4h_for_coin(df, base='ETH', prefix='ETH'):
    df = df.sort_values("date").reset_index(drop=True)
    df_ta = df.copy()

    # 데이터 매핑
    close = df[f"{base}_Close"]
    high = df.get(f"{base}_High", close)
    low = df.get(f"{base}_Low", close)
    volume = df.get(f"{base}_Volume", pd.Series(index=df.index, data=1))
    
    # 1. ATR (절대값) & NATR (비율)
    atr_14 = ta.atr(high, low, close, length=14)
    # df_ta[f"{prefix}_ATR_14"] = atr_14  <-- [삭제] 절대값은 위험함
    df_ta[f"{prefix}_NATR_14"] = (atr_14 / close) * 100 # [유지] 이건 아주 좋음

    # 2. Bollinger Bands
    bb = ta.bbands(close, length=20, std=2) 
    
    if bb is not None:
        bbl_col = [c for c in bb.columns if c.startswith("BBL")][0]
        bbu_col = [c for c in bb.columns if c.startswith("BBU")][0]
        bbm_col = [c for c in bb.columns if c.startswith("BBM")][0]


        # Bandwidth (변동성 폭) - 좋음
        df_ta[f"{prefix}_BB_WIDTH"] = ((bb[bbu_col] - bb[bbl_col]) / (bb[bbm_col] + 1e-9)) * 100
        
        # %B (상대적 위치) - 좋음 (0~1 사이 정규화됨)
        df_ta[f"{prefix}_BB_PCT"] = (close - bb[bbl_col]) / (bb[bbu_col] - bb[bbl_col] + 1e-9)


    # 3. Moving Averages (값 자체는 저장 X, 이격도만 저장)
    ema_20 = ta.ema(close, length=20)
    ema_50 = ta.ema(close, length=50)
    ema_200 = ta.ema(close, length=200)
    

    
    # [유지] 이격도(Distance)는 아주 훌륭한 피처임
    df_ta[f"{prefix}_Dist_EMA_20"] = (close / (ema_20 + 1e-9)) - 1
    df_ta[f"{prefix}_Dist_EMA_50"] = (close / (ema_50 + 1e-9)) - 1   # 추가 추천
    df_ta[f"{prefix}_Dist_EMA_200"] = (close / (ema_200 + 1e-9)) - 1 # 추가 추천
    
    # 4. RSI (0~100, 정상성 확보됨)
    df_ta[f"{prefix}_RSI_14"] = ta.rsi(close, length=14)
    
    
        # 5. MACD (정확하고 안전한 구현)
    macd = ta.macd(close, fast=12, slow=26, signal=9)
    
    if macd is not None and not macd.empty:
        cols = macd.columns.tolist()
        
        # 1. Histogram (히스토그램) 찾기 - 보통 'h'가 포함됨
        # 우선순위: 'MACDh'로 시작하는 것 -> 없으면 중간에 'h'가 있고 'MACD'인 것 -> 없으면 2번째 컬럼(iloc)
        hist_candidates = [c for c in cols if c.startswith("MACDh")]
        if not hist_candidates:
             hist_candidates = [c for c in cols if "hist" in c.lower()]
        
        if hist_candidates:
            hist_col = hist_candidates[0]
            df_ta[f"{prefix}_MACD_Hist"] = macd[hist_col]
        else:
            # 정 못 찾겠으면 순서상 2번째가 히스토그램임 (표준)
            df_ta[f"{prefix}_MACD_Hist"] = macd.iloc[:, 1]

        # 2. Signal Line (시그널) 찾기 - 보통 's'가 포함됨
        sig_candidates = [c for c in cols if c.startswith("MACDs")]
        if not sig_candidates:
            sig_candidates = [c for c in cols if "signal" in c.lower()]
            
        if sig_candidates:
            sig_col = sig_candidates[0]
            df_ta[f"{prefix}_MACD_Signal"] = macd[sig_col]
        else:
            # 정 못 찾겠으면 순서상 3번째가 시그널임
            df_ta[f"{prefix}_MACD_Signal"] = macd.iloc[:, 2]

        # 3. MACD Line (본선) 찾기 - 나머지 하나, 혹은 'MACD_'로 시작
        macd_candidates = [c for c in cols if c.startswith("MACD_") and c not in (hist_candidates + sig_candidates)]
        
        if macd_candidates:
            macd_col = macd_candidates[0]
            df_ta[f"{prefix}_MACD"] = macd[macd_col]
        else:
            # 정 못 찾겠으면 순서상 1번째가 본선임
            df_ta[f"{prefix}_MACD"] = macd.iloc[:, 0]


    # 5. ADX (추세 강도)
    adx = ta.adx(high, low, close, length=14)
    if adx is not None:
        # ADX도 첫번째 컬럼이 ADX 값임
        df_ta[f"{prefix}_ADX_14"] = adx.iloc[:, 0]

    # 6. Volume Ratio
    df_ta[f"{prefix}_Vol_Ratio"] = volume / (volume.rolling(20).mean() + 1e-9)

    return df_ta


# --------------------------------------------------------------------------------
# 3. 고급/복합 지표 함수 (Regime, VWAP, Institutional)
# --------------------------------------------------------------------------------
def add_volatility_regime_features_4h(df, prefix='ETH'):
    """Squeeze(수축) 및 고변동성 국면 판단"""
    df_ta = df.copy()
    
    # 1. High Volatility Regime
    atr_col = f'{prefix}_ATR_14'
    if atr_col in df_ta.columns:
        # 최근 120봉(20일) 평균보다 ATR이 높으면 고변동성
        atr_ma = df_ta[atr_col].rolling(120).mean()
        df_ta[f'{prefix}_high_volatility_regime'] = (df_ta[atr_col] > atr_ma).astype(int)
    
    # 2. Bollinger Band Squeeze (폭발 임박 신호)
    bbw_col = f'{prefix}_BB_WIDTH'
    if bbw_col in df_ta.columns:
        # 최근 120봉 중 하위 20% 폭이면 Squeeze
        bbw_threshold = df_ta[bbw_col].rolling(120, min_periods=20).quantile(0.2)
        df_ta[f'{prefix}_is_squeeze'] = (df_ta[bbw_col] < bbw_threshold).astype(int)
        
    return df_ta

def add_vwap_anchored_features_4h_for_coin(df, base='ETH', prefix='ETH'):
    """일일 VWAP 계산 및 이격도"""
    df_vwap = df.copy()

    # 날짜 컬럼 처리
    if "date" in df_vwap.columns:
        temp_date = pd.to_datetime(df_vwap["date"])
    else: 
        # date가 없으면 인덱스 사용 가정 혹은 에러 처리
        return df_vwap

    close = df_vwap[f"{base}_Close"]
    high = df_vwap.get(f"{base}_High", close)
    low = df_vwap.get(f"{base}_Low", close)
    volume = df_vwap.get(f"{base}_Volume", pd.Series(1, index=df_vwap.index))

    # Typical Price
    tp = (high + low + close) / 3
    pv = tp * volume

    # 일별 그룹화 (Daily VWAP Reset)
    day_series = temp_date.dt.date
    cum_pv = pv.groupby(day_series).cumsum()
    cum_vol = volume.groupby(day_series).cumsum()

    vwap = cum_pv / (cum_vol + 1e-9)
    
    # VWAP 이격도 (현재가가 VWAP보다 얼마나 위에 있나)
    df_vwap[f"{prefix}_Dist_from_VWAP"] = (close / vwap) - 1

    return df_vwap

def add_institutional_footprint_features_4h_for_coin(df, base='ETH', prefix='ETH'):
    """거래량 분석을 통한 세력 흔적 포착"""
    df_inst = df.copy()
    
    volume = df_inst.get(f"{base}_Volume", pd.Series(1, index=df_inst.index))
    close = df_inst[f"{base}_Close"]
    open_ = df_inst.get(f"{base}_Open", close)
    high = df_inst.get(f"{base}_High", close)
    low = df_inst.get(f"{base}_Low", close)

    # 1. 거래량 폭발 (Volume Spike)
    vol_ma = volume.rolling(20).mean()
    # 평균보다 2배 이상 거래량 터짐
    df_inst[f"{prefix}_Vol_Spike"] = (volume > (vol_ma * 2.0)).astype(int)

    # 2. 단봉 (Small Body) - 매집/분산 징후
    body_size = (close - open_).abs()
    candle_range = high - low
    # 몸통이 전체 길이의 30% 미만 (도지형 캔들)
    df_inst[f"{prefix}_Small_Body"] = (body_size < (candle_range * 0.3)).astype(int)

    # 3. Absorption (매집)
    # 거래량은 터졌는데 몸통이 작음 -> 누군가 다 받아냄
    df_inst[f"{prefix}_Absorption"] = (df_inst[f"{prefix}_Vol_Spike"] & df_inst[f"{prefix}_Small_Body"]).astype(int)

    return df_inst





2025-12-05 10:58:59.284214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-05 10:58:59.289346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-05 10:58:59.290979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-05 10:58:59.304569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-05 10:59:00.454553: W tensorflow/comp

In [2]:
# --------------------------------------------------------------------------------
# 5. 기타 파생변수 생성 함수 
# --------------------------------------------------------------------------------
def calculate_cumulative_volume_delta_4h(df):
    df_cvd = df.copy()
    close = df[f"{base}_Close"]
    high = df.get(f"{base}_High", close)
    low = df.get(f"{base}_Low", close)
    volume = df.get(f"{base}_Volume", pd.Series(index=df.index, data=1))
    open_ = df.get(f"{base}_Open", close)
    

    vol_delta_series = np.where(close > open_, volume, -volume)
    vol_delta_series = pd.Series(vol_delta_series, index=df.index)
    
    cvd_series = vol_delta_series.cumsum()   
    
    cvd_24h_rolling = vol_delta_series.rolling(6).sum()
    df_cvd[f'{prefix}_CVD_Rank_180'] = cvd_24h_rolling.rolling(180).rank(pct=True)

    cvd_slope = cvd_series.diff(6)
    price_volatility = close.rolling(24).std() + 1e-9
    df_cvd[f'{prefix}_CVD_Slope_Norm'] = cvd_slope / price_volatility 
    
    
    return df_cvd


import pandas as pd
import numpy as np

def add_vwma_features_4h(df, base, prefix=None):
    """
    base  : 'BTC', 'ETH', 'XRP' 같은 코인 코드
    prefix: 생성할 컬럼 접두어 (기본은 base와 동일)
    """
    if prefix is None:
        prefix = base

    close_col = f"{base}_Close"
    vol_col   = f"{base}_Volume"   # Binance 쓰고 싶으면 f"{base}_Bin_Volume" 등으로 바꿔도 됨

    if close_col not in df.columns:
        return df

    close = df[close_col]
    if vol_col in df.columns:
        volume = df[vol_col]
    else:
        volume = pd.Series(1.0, index=df.index)

    for period in [20, 50]:
        window = period * 6

        num = (close * volume).rolling(window=window, min_periods=1).sum()
        den = volume.rolling(window=window, min_periods=1).sum() + 1e-9
        vwma = num / den

        df[f"{prefix}_Price_div_VWMA_{period}d"] = (close / (vwma + 1e-9)) - 1

    return df


def add_obv_divergence_features_4h_for_coin(df, base, prefix):
    df_obv = df.copy()
    close = df_obv[f"{base}_Close"]
    volume = df_obv.get(f"{base}_Volume", pd.Series(index=df_obv.index, data=1))

    obv = ta.obv(close, volume)

    obv_ma_30 = obv.rolling(30).mean()
    obv_ma_60 = obv.rolling(60).mean()
    df_obv[f"{prefix}_OBV_Trend"] = np.where(obv_ma_30 > obv_ma_60, 1, -1)

    price_slope = close.diff(12)
    obv_slope = obv.diff(12)

    df_obv[f"{prefix}_Div_Bullish"] = ((price_slope < 0) & (obv_slope > 0)).astype(int)
    df_obv[f"{prefix}_Div_Bearish"] = ((price_slope > 0) & (obv_slope < 0)).astype(int)

    return df_obv


def add_vwap_anchored_features_4h_for_coin(df, base, prefix):
    df_vwap = df.copy()

    if "date" not in df_vwap.columns and "timestamp" in df_vwap.columns:
        temp_date = pd.to_datetime(df_vwap["timestamp"])
    else:
        temp_date = pd.to_datetime(df_vwap["date"])

    close = df_vwap[f"{base}_Close"]
    high = df_vwap.get(f"{base}_High", close)
    low = df_vwap.get(f"{base}_Low", close)
    volume = df_vwap.get(f"{base}_Volume", pd.Series(1, index=df_vwap.index))

    typical_price = (high + low + close) / 3

    day_series = temp_date.dt.date
    pv = typical_price * volume

    cum_pv = pv.groupby(day_series).cumsum()
    cum_vol = volume.groupby(day_series).cumsum()

    vwap_day_series = cum_pv / (cum_vol + 1e-9)

    df_vwap[f"{prefix}_Dist_from_VWAP"] = (close / vwap_day_series) - 1

    return df_vwap


def add_institutional_footprint_features_4h_for_coin(df, base, prefix):
    df_inst = df.copy()
    volume = df_inst.get(f"{base}_Volume", pd.Series(index=df_inst.index, data=1))
    close = df_inst[f"{base}_Close"]
    open_ = df_inst.get(f"{base}_Open", close)
    high = df_inst.get(f"{base}_High", close)
    low = df_inst.get(f"{base}_Low", close)

    vol_ma = volume.rolling(30).mean()
    df_inst[f"{prefix}_Vol_Spike"] = (volume > (vol_ma * 2.0)).astype(int)

    body_size = (close - open_).abs()
    candle_range = high - low
    df_inst[f"{prefix}_Small_Body"] = (body_size < (candle_range * 0.3)).astype(int)

    df_inst[f"{prefix}_Absorption"] = (df_inst[f"{prefix}_Vol_Spike"] & df_inst[f"{prefix}_Small_Body"]).astype(int)

    lower_wick = (open_.combine(close, min) - low)
    upper_wick = (high - open_.combine(close, max))

    df_inst[f"{prefix}_Buying_Climax"] = (
        df_inst[f"{prefix}_Vol_Spike"] & (upper_wick > body_size * 2)
    ).astype(int)
    df_inst[f"{prefix}_Selling_Climax"] = (
        df_inst[f"{prefix}_Vol_Spike"] & (lower_wick > body_size * 2)
    ).astype(int)

    return df_inst


def add_cyclical_time_features_4h(df):
    df_time = df.copy()
    
    # timestamp가 datetime 객체라고 가정
    if not pd.api.types.is_datetime64_any_dtype(df_time['date']):
        df_time['date'] = pd.to_datetime(df_time['date'])
        
    # 1. 시간 (Hour) - 4시간 봉이므로 0, 4, 8, 12, 16, 20 등의 값이 나옴
    # 하루는 24시간이므로 주기는 24
    df_time['hour_sin'] = np.sin(2 * np.pi * df_time['date'].dt.hour / 24)
    df_time['hour_cos'] = np.cos(2 * np.pi * df_time['date'].dt.hour / 24)
    
    # 2. 요일 (Day of Week) - 0(월) ~ 6(일)
    # 일주일은 7일이므로 주기는 7
    df_time['day_sin'] = np.sin(2 * np.pi * df_time['date'].dt.dayofweek / 7)
    df_time['day_cos'] = np.cos(2 * np.pi * df_time['date'].dt.dayofweek / 7)
    
    # 3. 월 (Month) - 계절성이나 연말 효과 반영
    df_time['month_sin'] = np.sin(2 * np.pi * df_time['date'].dt.month / 12)
    df_time['month_cos'] = np.cos(2 * np.pi * df_time['date'].dt.month / 12)
    
    return df_time

In [3]:
def remove_raw_prices_and_transform(df, save_path="eth_4hour_raw_prices.csv"):
    """
    1. 삭제될 Raw Price 컬럼 식별
    2. 삭제될 컬럼들만 따로 csv로 저장 (백업) - ★복구됨★
    3. 원본 df에서 해당 컬럼 삭제 후 반환
    """
    df_transformed = df.copy()
    
    # (1) 삭제할 패턴 정의
    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    
    # 보호할 키워드 (핵심 지표들)
    keep_keywords = [
        '_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv',
        'PRICE_VS', 'BREAKOUT', 'UPPER_SHADOW', 'LOWER_SHADOW', 'BB_WIDTH', 'KP', 'FR', 'RelStr',
        'CVD', 'VWMA', 'VWAP', 'Div', 'Spike', 'Climax', 'Absorption', 
        'is_squeeze', 'regime', 'signal', 'trend' 
    ]
    
    # (2) 삭제 대상 식별
    cols_to_remove = []
    for col in df_transformed.columns:
        if any(p in col for p in remove_patterns):
            # 보호 키워드가 있으면 삭제 안 함
            if any(k.lower() in col.lower() for k in keep_keywords):
                continue 
            else:
                cols_to_remove.append(col)

    # (3) ★ 중요: 삭제될 컬럼들 백업 (CSV 저장) ★
    if cols_to_remove:
        print(f" Saving {len(cols_to_remove)} raw price columns to {save_path}...")
        
        # 날짜 컬럼이 있으면 같이 저장해서 나중에 매칭하기 쉽게 함
        if 'date' in df_transformed.columns:
            save_cols = ['date'] + cols_to_remove
        elif 'timestamp' in df_transformed.columns:
            save_cols = ['timestamp'] + cols_to_remove
        else:
            save_cols = cols_to_remove
            
        # 해당 컬럼들만 뽑아서 저장
        df_transformed[save_cols].to_csv(save_path, index=False)
        print(f" Backup complete. Columns saved: {cols_to_remove[:3]} ...")

    # (4) 원본에서 삭제
    print(f" Removing raw price columns from main dataframe...")
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)
    
    return df_transformed



def preprocess_non_stationary_features_4h(df):
    """
    비정상성 데이터 전처리 함수 (수정됨: 이진 변수 보호 강화 + Rolling Z-Score 적용)
    """
    df_proc = df.copy()

    # 1. 절대 변환하거나 삭제하지 말아야 할 '이진 변수(0/1)' 및 '핵심 지표' 목록
    whitelist_keywords = [
        "is_squeeze", "absorption", "regime", "signal", "trend", 
        "climax", "spike", "div_", "obv_trend", "regime"
    ]

    # 2. [수정됨] 반드시 정규화(Z-Score) 해야 할 변수들 (MACD, ADX, NATR 추가됨)
    must_transform_keywords = [
        "sp500", "gold", "dxy", "vix", "usdt_eth_mcap",
        "tvl", "mcap", "chain", "lido", "uniswap", "aave", "makerdao",
        "curve", "optimism", "arbitrum", "base", "zksync",
        "ema_", "sma_", "obv", 
        "macd", "adx", "natr"  # <-- ★ 추가된 부분 ★
    ]

    kp_cols = [
        col for col in df_proc.columns
        if "_kp_" in col.lower()
        and "change" not in col.lower()
        and "zscore" not in col.lower()
        and "slope" not in col.lower()
    ]

    # KP 관련 처리 (기존 유지)
    for col in kp_cols:
        series = df_proc[col].fillna(method="ffill").replace(0, 1e-9)
        if not (series <= 0).any():
            df_proc[f"{col}_log"] = np.log(series + 1e-9)
        df_proc[f"{col}_slope_180"] = (
            series - series.rolling(180).mean()
        ) / (series.rolling(180).mean() + 1e-9)

    cols_to_drop = []

    for col in df_proc.columns:
        col_lower = col.lower()

        # 화이트리스트 패스
        if any(w in col_lower for w in whitelist_keywords):
            continue

        # 기존 보호 로직
        if col in kp_cols or any(x in col_lower for x in ["_log", "_slope"]):
            continue

        if any(x in col_lower for x in ["_pct", "_ratio", "_chg", "_diff", "zscore", "rate", "fear"]):
            continue

        if col_lower in ["date", "timestamp", "open", "high", "low", "close", "volume"]:
            continue
            
        # 3. [핵심 수정] 변환 대상 키워드 처리 (Z-Score 적용)
        if any(k in col_lower for k in must_transform_keywords):
            series = df_proc[col].fillna(method="ffill").replace(0, 1e-9)
            
            # (1) 24시간(6봉) 변화율 생성 (기존 유지)
            df_proc[f"{col}_pct_chg_6"] = series.pct_change(6).fillna(0)
            
            # (2) ★ Rolling Z-Score로 변경 ★
            # 기존: ((series - ma) / ma) -> 비율 방식 (분산 반영 안됨)
            # 변경: ((series - ma) / std) -> 표준편차 방식 (시장 상황 적응형)
            roll_mean = series.rolling(window=180, min_periods=60).mean()
            roll_std = series.rolling(window=180, min_periods=60).std()
            
            df_proc[f"{col}_zscore_180"] = ((series - roll_mean) / (roll_std + 1e-9)).fillna(0)
            
            # 원본은 삭제 리스트에 추가
            cols_to_drop.append(col)

    if cols_to_drop:
        print(f" [Preprocessing] Dropping {len(cols_to_drop)} non-stationary features...")
        df_proc.drop(columns=cols_to_drop, inplace=True)

    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    return df_proc

In [4]:



# =============================================================================
# 2. 메인 실행 코드
# =============================================================================

# GPU Check (Optional)
if tf.config.list_physical_devices('GPU'):
    print("GPU Detected!")
else:
    print(" Running on CPU")

print("Loading Data...")
df_merged = pd.read_csv("macro_data_4h/eth_4hour.csv")

rename_map = {}
for c in ["BTC","ETH","XRP","SOL","ADA","DOGE","AVAX","DOT"]:
    old = f"{c}_Bin_Vol"
    new = f"{c}_Bin_Volume"
    if old in df_merged.columns:
        rename_map[old] = new

df_merged = df_merged.rename(columns=rename_map)


# 날짜 처리
if 'timestamp' in df_merged.columns:
    df_merged = df_merged.rename(columns={'timestamp': 'date'})
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged = df_merged.sort_values('date').reset_index(drop=True)

# 작업 시작
df = df_merged.copy()

# 피처 엔지니어링 (순차 실행)
print("Generating Features...")

# COINS = ["BTC","ETH","XRP","SOL","ADA","DOGE","AVAX","DOT"]
COINS=["ETH"]

# 1) 원시가격 컬럼들 정확히 지정
raw_cols = ["date"]
for c in COINS:
    raw_cols += [
        f"{c}_Open", f"{c}_High", f"{c}_Low", f"{c}_Close", f"{c}_Volume",
        f"{c}_Bin_Open", f"{c}_Bin_High", f"{c}_Bin_Low", f"{c}_Bin_Close",
        f"{c}_Bin_Volume"
    ]

df_raw = df_merged[raw_cols].copy()
df_raw.to_csv("eth_4hour_raw_price.csv", index=False)

df = df_merged.copy()


# 코인별 TA / 모멘텀
for c in COINS:
    df = calculate_technical_indicators_4h_for_coin(df, base=c, prefix=c)
    df = add_vwma_features_4h(df, base=c, prefix=c)
    df = add_obv_divergence_features_4h_for_coin(df, base=c, prefix=c)
    df = add_vwap_anchored_features_4h_for_coin(df, base=c, prefix=c)
    df = add_institutional_footprint_features_4h_for_coin(df, base=c, prefix=c)
# 시장 공통 피처들...
df = add_kimchi_premium_proxy(df)
df = add_funding_and_onchain_features(df)
df = add_cyclical_time_features_4h(df)
df=  add_volatility_regime_features_4h(df)
df=  add_cyclical_time_features_4h(df)

df = preprocess_non_stationary_features_4h(df)
df.to_csv("eth_4_all.csv",index=False)

df = df.replace([np.inf, -np.inf], np.nan).fillna(0)


df_clean = remove_raw_prices_and_transform(df, save_path="eth_4_price.csv")
df_clean.to_csv("eth_4.csv", index=False)
print(f"✅ All Done! Saved  shape: {df_clean.shape}")

GPU Detected!
Loading Data...
Generating Features...


/tmp/ipykernel_2537528/76279209.py:122: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ema_20 = ta.ema(close, length=20)
/tmp/ipykernel_2537528/76279209.py:123: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ema_50 = ta.ema(close, length=50)
/tmp/ipykernel_2537528/76279209.py:124: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ema_200 = ta.ema(close, length=200)


 [Preprocessing] Dropping 22 non-stationary features...
 Saving 40 raw price columns to eth_4_price.csv...
 Backup complete. Columns saved: ['BTC_Open', 'BTC_High', 'BTC_Low'] ...
 Removing raw price columns from main dataframe...
✅ All Done! Saved  shape: (17936, 141)


In [5]:
df_ta=df_clean.copy()
bbw_col = f'ETH_BB_WIDTH'
if bbw_col in df_ta.columns:
    # 최근 120봉 중 하위 20% 폭이면 Squeeze
    bbw_threshold = df_ta[bbw_col].rolling(120, min_periods=20).quantile(0.2)
    df_ta[f'ETH_is_squeeze'] = (df_ta[bbw_col] < bbw_threshold).astype(int)
print(df_ta.shape)
df_ta.to_csv("eth_4hour_ETH.csv",index=False)


(17936, 141)


In [6]:
print(df_clean.select_dtypes(include='bool').columns)

Index([], dtype='object')
